#  Portfolio Optimization

In this problem, we will find the portfolio allocation that minimizes risk while achieving a given expected return $R_\mbox{target}$.

Suppose that we know the mean returns $\mu \in \mathbf{R}^n$ and the covariance $\Sigma \in \mathbf{R}^{n \times n}$ of the $n$ assets. We would like to find a portfolio allocation $w \in \mathbf{R}^n$, $\sum_i w_i = 1$, minimizing the *risk* of the portfolio, which we measure as the variance $w^T \Sigma w$ of the portfolio. The requirement that the portfolio allocation achieve the target expected return can be expressed as $w^T \mu >= R_\mbox{target}$. We suppose further that our portfolio allocation must comply with some lower and upper bounds on the allocation, $w_\mbox{lower} \leq w \leq w_\mbox{upper}$.

This problem can be written as

$
\begin{array}{ll}
    \mbox{minimize}   & w^T \Sigma w \\
    \mbox{subject to} & w^T \mu >= R_\mbox{target} \\
                      & \sum_i w_i = 1 \\
                      & w_\mbox{lower} \leq w \leq w_\mbox{upper}
\end{array}
$

where $w \in \mathbf{R}^n$ is our optimization variable.

In [1]:
using Convex, SCS

# generate problem data
μ = [11.5; 9.5; 6]/100          #expected returns
Σ  = [166  34  58;              #covariance matrix
       34  64   4;
       58   4 100]/100^2

n = length(μ)                   #number of assets 

R_target = 0.1
w_lower = 0
w_upper = 0.5;

If you want to try the optimization with more assets, uncomment and run the next cell. It creates a vector or average returns and a variance-covariance matrix that have scales similar to the numbers above.

In [2]:
#=
using Random
Random.seed!(123)

n = 15                                      #number of assets, CHANGE IT?

μ = (6 .+ (11.5-6)*rand(n))/100             #mean
A = randn(n,n)
Σ = (A * A' + diagm(0=>rand(n)))/500;       #covariance matrix
=#

In [2]:
w    = Variable(n)
ret  = dot(w,μ)
risk = quadform(w,Σ)

p = minimize( risk, 
              ret >= R_target, 
              sum(w) == 1, 
              w_lower <= w, 
              w <= w_upper )

solve!(p, SCSSolver())     #use SCSSolver(verbose = false) to suppress printing

----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 27, CG tol ~ 1/iter^(2.00)
eps = 1.00e-005, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-003
Variables n = 6, constraints m = 17
Cones:	primal zero / dual free vars: 2
	linear vars: 8
	soc vars: 7, soc blks: 2
Setup time: 4.89e-005s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|3.46e+019 2.62e+019 1.00e+000 -6.54e+019 7.73e+018 1.06e+019 2.73e-005 
    60|2.16e-008 5.74e-008 7.46e-008 6.32e-003 6.32e-003 1.21e-016 2.45e-004 
--------------------------------------

In [3]:
display("Optimal portfolio weights")
display(w.value)

"Optimal portfolio weights"

3×1 Array{Float64,2}:
 0.40909103267633684
 0.5000000044451999 
 0.09090892365112054

In [4]:
sum(w.value)

0.9999999607726573